# Adjust info

This notebook is used to consider late entries, reschedule matches, and move players from and into subteams.

In [15]:
from helper_functions import setup
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import helper_functions as hf
from helper_functions.setup.team_creation import find_best_team_to_join
from helper_functions.setup.openai_image_download import generate_all_images, save_resized_animal_images
# player_df = setup.sanitize_and_anonymize_data(overwrite=False, verbose=False, anonymize=False)
# player_df[player_df["email"].str.contains("3")][["name", "email"]]
# hf.ALL_SUBTEAMS


## Register latecomers

Add all latecoming players, and add them as a reserve in each subteam for the best-fitting team.

In [18]:
player_df = setup.sanitize_and_anonymize_data(overwrite=True, verbose=False, anonymize=True)
late = player_df[player_df["late_entry"]]
teams = hf.get_teams()
subteams = hf.ALL_SUBTEAMS.values()
teams_to_join = {"Thorny Warthog": "C",
                 "Defenseless Insect": "B",
                 "Querulous Piranha": "B",
                 "Spanish Ferret": "A",
                "Euphoric Vulture": "B",
                "Staid Hornbill": "B",
                "Illiterate Caterpillar": "C",
                "Probable Herring": "C",
                "Worthy Aurochs": "B",
                "Miserable Alligator": "A",
                "Gripping Penguin": "B",
                "Infamous Sawfish": "C",
                "Worldly Discus": "A",
                "Unimportant Beetle": "C",
                "Inconsequential Bandicoot": "B",
                "Deficient Squid": "A",
                "Honored Tamaskan": "C",
                "Athletic Carp": "B",
                "Glorious Mule": "A",
                "Soulful Ant": "A",
                "Grandiose Orang-utan": "B",
                "Elastic Pekingese": "A",
                  }

for _, player in late.iterrows():
    name = player["nickname"]
    # Don't do anything if the player has already been added
    if any([team.contains_player(name) for team in teams]):
        continue
    if name in teams_to_join:
        # team_to_join = teams[find_best_team_to_join(teams, player)]
        team_to_join = teams["ABC".index(teams_to_join[name])]
        team_to_join.add_player(player, register_as_reserve=True)
        print(f"{name} joins {team_to_join.name}")
        generate_all_images([name])
# save_resized_animal_images(200)

def _rewrite_teams():
    old_df = hf.get_players()
    player_df = setup.sanitize_and_anonymize_data(overwrite=False, verbose=False, anonymize=True)
    cols = [col for col in old_df.columns if col != "confirmation_status"]
    old_df = pd.merge(old_df[cols], player_df[["nickname", "confirmation_status"]], on="nickname")
    for team_letter in "ABC":
        fpath = hf.Team.backup_path_from_letter(team_letter)
        old_df[old_df["Team"] == f"Team {team_letter}"].to_csv(fpath, index=False)
_rewrite_teams()
# print("\n\n".join(player_df.iloc[-10:]["nickname"]))
player_df.tail()


,nickname,institute,confirmation_status,is_postdoc,avail_monday,avail_tuesday,avail_thursday,avail_friday,wants_ping_pong,ping_pong,...,capture_the_flag,wants_spikeball,spikeball,wants_beer_pong,beer_pong,wants_foosball,foosball,num_sports,num_sports_not_avail,late_entry
111,Athletic Carp,MPE,True,True,False,False,True,True,True,True,...,True,True,True,False,False,False,False,3,0,True
112,Glorious Mule,MPA,True,False,False,False,True,False,False,False,...,True,False,False,False,False,False,False,1,0,True
113,Soulful Ant,MPE,True,False,False,False,True,False,False,False,...,False,True,True,False,False,False,False,1,0,True
114,Grandiose Orang-utan,MPE,True,False,False,False,True,False,False,False,...,True,False,False,False,False,False,False,1,0,True
115,Elastic Pekingese,MPE,True,False,False,False,False,True,True,True,...,False,False,False,False,False,False,False,1,0,True


## Move players from subteam to subteam

If players are attending a sport, even if just in reserve, and are substituted in, they should be registered there.

In [29]:
# df.loc["Fine Oyster", "subteam_football"] = "R"
# df.loc["Lone Rhinoceros", "subteam_football"] = "R"
def set_player_subteam(nickname: str, sport: str, new_subteam: str, verbose: bool = True):
    """Load the table of players and set the subteam to the desired one"""
    sport = sport.lower()
    df = hf.get_players().set_index("nickname", drop=False).fillna("")
    old_subteam = df.loc[nickname, f"subteam_{sport}"]
    player_team = df.loc[nickname, "Team"]
    print(player_team)
    if old_subteam == new_subteam:
        if verbose:
            print(f"New subteam and old subteam '{old_subteam}' are the same.")
        return
    log_msg = f"Moving '{nickname}' ({player_team}) from '{old_subteam}' to '{new_subteam}' in {sport}."
    if verbose:
        print(log_msg)
    if (new_subteam != "") and (new_subteam not in np.unique(df[f"subteam_{sport}"])):
        print(f"Registering an unknown new subteam {new_subteam} for {sport}")
    df.loc[nickname, f"subteam_{sport}"] = new_subteam
    for team_letter in "ABC":
        fpath = hf.Team.backup_path_from_letter(team_letter)
        df[df["Team"] == f"Team {team_letter}"].to_csv(fpath, index=False)
    with hf.DATAPATH.joinpath("teams/changelog.txt").open("a", encoding="utf-8") as f:
        f.write(log_msg + "\n")


# hf.get_players().set_index("nickname").loc["Qualified Buffalo"]
# set_player_subteam("Rewarding Ermine", "capture_the_flag", "")
set_player_subteam("Grandiose Orang-utan", "ping_pong", "21")


# hf.get_players().set_index("nickname").loc["Wide Lionfish"]


Team B
Moving 'Grandiose Orang-utan' (Team B) from '' to '21' in ping_pong.
Registering an unknown new subteam 21 for ping_pong


## Move players from one team to another

In [4]:
def set_player_team(nickname: str, new_team: str, verbose: bool = True):
    """Load the table of players and set the subteam to the desired one"""
    assert new_team in [f"Team {letter}" for letter in "ABC"], f"{new_team} is not a valid team."
    df = hf.get_players().set_index("nickname", drop=False).fillna("")
    old_team = df.loc[nickname, "Team"]
    if old_team == new_team:
        if verbose:
            print(f"New team and old team '{old_team}' are the same.")
        return
    log_msg = f"Moving '{nickname}' from '{old_team}' to '{new_team}'."
    old_subteams = {sport: subteam for sport in hf.SPORTS_LIST 
                    if (subteam := df.loc[nickname, f"subteam_{sport}"]) != ""}
    log_msg += f" This affects the subteams {old_subteams}"
    if verbose:
        print(log_msg)
    df.loc[nickname, "Team"] = new_team
    for team_letter in "ABC":
        fpath = hf.Team.backup_path_from_letter(team_letter)
        df[df["Team"] == f"Team {team_letter}"].to_csv(fpath, index=False)
    with hf.DATAPATH.joinpath("teams/changelog.txt").open("a", encoding="utf-8") as f:
        f.write(log_msg + "\n")

set_player_team("Worthy Aurochs", "Team A")


New team and old team 'Team A' are the same.


## Reschedule Matches

Some matches need to be rescheduled. We can do that here.

In [5]:
from datetime import datetime, time
from helper_functions.setup.match_scheduling import write_match_backup_from_df
from itertools import combinations

def reschedule_match_to_new_time(match_key: str, new_time: datetime | time, check_overlaps: bool = False):
    matches = hf.get_match_df().set_index("full_key", drop=False)
    old_time = matches.loc[match_key, "start"]
    if isinstance(new_time, time):
        new_time = datetime.combine(old_time, new_time)  # type: ignore
        print(new_time)
    matches.loc[match_key, "start"] = new_time
    matches.loc[match_key, "day"] = new_time.strftime("%A")
    matches.loc[match_key, "time"] = new_time.strftime("%H:%M")
    print(f"Rewritten the time for {match_key} to {new_time.strftime("%H:%M (%A)")}.")
    write_match_backup_from_df(matches, True)
    if check_overlaps:
        matches = hf.get_matches()
        num_conflicts = np.sum([comb[0].has_hard_collision(comb[1], verbose=True) for comb in combinations(matches, 2)])
        if num_conflicts == 0:
            print("No conflicts found.")

# reschedule_match_to_new_time("ping_pongB: 14C: 16", datetime(2024, 4, 30, 13, 30))
# df = hf.turn_series_list_to_dataframe([m.as_series for m in hf.ALL_MATCHES])
# df[df["sport"] == "ping_pong"]


Rewritten the time for ping_pongB: 14C: 16 to 13:30 (Tuesday).


In [4]:
for sport in hf.SPORTS_EVENTS.values():
    print(f"{sport.name}: {sport.single_match_win_value:.2f}")

def register_or_add_to_dict(d: dict[str, float], key: str, val: float):
    """Registers the value with the given key in the dict if not present,
    or adds it to the existing value if already there.
    Therefore mutates the dict!
    """
    if key in d:
        d[key] += val
    else:
        d[key] = val

winner_dict: dict[str, float] = dict()
for match in hf.ALL_MATCHES:
    win_val = hf.SPORTS_EVENTS[match.sport].single_match_win_value
    for win_player in match.winning_players:
        register_or_add_to_dict(winner_dict, win_player, win_val)
    for tie_player in match.tying_players:
        register_or_add_to_dict(winner_dict, tie_player, win_val/2)

for team_letter in "ABC":
    win_val = {"B": 35, "C": 34, "A": 31}[team_letter]/hf.SPORTS_EVENTS["running_sprints"].num_players_per_subteam
    for player in hf.ALL_SUBTEAMS[f"running_sprints_{team_letter}: 1"].players:
        register_or_add_to_dict(winner_dict, player, win_val)

players = hf.get_players().infer_objects(True).fillna("")
players["score"] = players["nickname"].apply(lambda x: winner_dict.get(x, 0))
# print(players.columns)
players["sports"] = players.apply(lambda x: [sport for sport in hf.SPORTS_LIST if x[f"subteam_{sport}"] not in ["", "R"]], axis=1)
players[["nickname", "Team", "score", "sports"]].sort_values("score", ascending=False).reset_index().reset_index(drop=True).head(10)#.set_index("nickname", drop=False).loc["Limp Monkey"]



Ping Pong: 4.90
Basketball: 8.33
Running and Sprints: 8.33
Volleyball: 6.25
Chess: 11.11
Football: 6.25
Tennis: 11.11
Capture the flag: 6.25
Spikeball: 6.25
Beer Pong: 5.56
Foosball: 6.25


,index,nickname,Team,score,sports
0,8,Trifling Tarpon,Team A,52.124183,"[ping_pong, volleyball, chess, football]"
1,0,Trivial Uguisu,Team B,47.679739,"[ping_pong, basketball, running_sprints, chess..."
2,2,Radiant Booby,Team B,43.443627,"[ping_pong, volleyball, football, tennis, spik..."
3,9,Pushy Bulldog,Team C,42.790033,"[ping_pong, volleyball, football, tennis, spik..."
4,2,Cool Weasel,Team A,41.013072,"[ping_pong, volleyball, chess, football]"
5,0,Magnificent Barracuda,Team A,38.305556,"[ping_pong, running_sprints, football, tennis,..."
6,23,Pleasing Mongrel,Team A,34.722222,"[chess, capture_the_flag]"
7,9,Dramatic Hyena,Team A,32.750000,"[running_sprints, football, capture_the_flag]"
8,1,Thankful Kakapo,Team B,27.928922,"[ping_pong, running_sprints, football, capture..."
9,3,Minty Possum,Team C,25.901961,"[ping_pong, running_sprints, capture_the_flag,..."


### Add more matches

For ping pong, I wanted to register some matches between a few latecomers.

In [14]:
from helper_functions.setup.match_scheduling import write_match_backup
from datetime import datetime, timedelta
ppA = hf.ALL_SUBTEAMS["ping_pong_A: 17"]
ppB = hf.ALL_SUBTEAMS["ping_pong_B: 17"]
ppC = hf.ALL_SUBTEAMS["ping_pong_C: 17"]
m1 = hf.Match("ping_pong", datetime(2024, 5, 2, 18, 15), timedelta(minutes=30), ppA, ppC, "1")
m2 = hf.Match("ping_pong", datetime(2024, 5, 2, 18, 45), timedelta(minutes=30), ppB, ppC, "1")
m3 = hf.Match("ping_pong", datetime(2024, 5, 2, 19, 15), timedelta(minutes=30), ppA, ppB, "1")
hf.ALL_MATCHES
new_matches = hf.ALL_MATCHES + [m1, m2, m3]
write_match_backup(new_matches, overwrite=True)


In [48]:
match_df = hf.turn_series_list_to_dataframe([m.as_series for m in hf.ALL_MATCHES])
# from random import choices, seed
# seed(42)
# match_df["subteam_a"] = match_df["team_a"]
# match_df["subteam_b"] = match_df["team_b"]
# match_df["team_a"] = match_df["team_a_key"].str.slice(0, 1)
# match_df["team_b"] = match_df["team_b_key"].str.slice(0, 1)
# match_df["winner"] = choices(["A", "B", "C", "AB", "AC", "BC"], k=len(match_df))

def _calc_score_for_team(match_df: pd.DataFrame, team_letter: str, sport: str) -> float:
    assert sport in hf.SPORTS_LIST
    assert team_letter in "ABC"
    sport_df = match_df[match_df["sport"] == sport].infer_objects(copy=False).fillna("")
    num_won = np.sum(sport_df["winner"] == team_letter)
    num_tied = np.sum(sport_df["winner"].apply(lambda x:  len(x) > 1 and team_letter in x))
    return num_won + 0.5 * num_tied

sport_nums = {sport: [_calc_score_for_team(match_df, team, sport) for team in "ABC"] for sport in hf.SPORTS_LIST}
sport_nums["Team"] = ["A", "B", "C"]
df = pd.DataFrame(sport_nums).set_index("Team")
for i, sport in enumerate(hf.SPORTS_LIST):
    event = hf.SPORTS_EVENTS[sport]
    result_perc = np.round(df[sport]/df[sport].sum() * 100, 0)*event.point_weight_factor
    colname = f"{event.icon} Score (x {event.point_weight_factor:.1f})"
    df.insert(i*2+1, colname, result_perc)
    # df[f"{sport}_perc"] = result_perc
    # df = df.drop(sport, axis=1)
score_df = df[[col for col in df.columns if "Score" in col]].copy()
score_df["Total Score"] = score_df.sum(axis=1)/score_df.sum(axis=0).sum()*500
import altair as alt


# Transform the DataFrame into a long format
score_df = score_df.reset_index().melt('Team', var_name='Sport', value_name='Score')

total_scores = score_df[score_df['Sport'] != 'Total Score'].groupby('Team')['Score'].sum()


# Add the total score for each team to the DataFrame
for team, total_score in total_scores.items():
    new_row = pd.DataFrame({'Team': [team], 'Sport': [f'Total (rescaled) Score {team}'], 'Score': [total_score]})
    score_df = pd.concat([score_df, new_row], ignore_index=True)
score_df = score_df[score_df['Sport'] != 'Total Score']

team_colors = alt.Color(
    "Team:N", scale=alt.Scale(range=[team.color for team in hf.ALL_TEAMS])
)
# Create the horizontal stacked bar plot
chart = alt.Chart(score_df).mark_bar().encode(
    y='Sport:N',
    x='Score:Q',
    color=team_colors,
    order=alt.Order(
      # Sort the segments of the bars by this field
      'Team:N',
    ),
    opacity=alt.value(0.5),
)

chart
# score_df.head()


alt.Chart(...)

In [ ]:

def get_email_text_for_player(player: pd.Series) -> tuple[str, str, Path]:
    email_base_text = hf.DATAPATH.joinpath("helper_texts/email_blank_text.md").read_text(encoding="utf-8")
    first_name = player["name"].split()[0]
    nickname = player["nickname"]
    team_name = player["Team"]
    cloth_color = {"Team A": "dark", "Team B": "white", "Team C": "colorful"}[team_name]
    email = get_email_address(player["email"])
    signal_link = hf.DATAPATH.joinpath("hidden/signal_entry_link.txt").read_text(encoding="utf-8")
    
    player_obj = Player.from_series(player, list(hf.ALL_MATCHES))
    schedule = player_obj.get_schedule_for_mail()
    fpath = hf.DATAPATH.joinpath(f"assets/animal_pics/full_size/{player["nickname"].lower().replace(" ", "_")}.png")
    text = email_base_text.format(first_name=first_name, nickname=nickname, schedule=schedule, team_name=team_name, cloth_color=cloth_color, signal_link=signal_link)
    html_text = markdown.markdown(text.replace("\\", "<br>"))
    return email, html_text, fpath

# player = df.loc["Thankful Kakapo"]
# address, text, fpath = get_email_text_for_player(player)
email, text, fpath = get_email_text_for_player(df.loc["Worldly Discus"])
print(email, "\n", fpath.name, "\nSports week (Apr 29 - May 3): Your schedule and information")
hf.copy_to_clipboard(text)


rydellet@MPA-Garching.MPG.DE 
 worldly_discus.png 
Sports week (Apr 29 - May 3): Your schedule and information


## Changing match locations

For some matches, it might make sense to move them around if they are back-to-back, but currently scheduled at different courts.

In [90]:

def change_match_location(match_key: str, new_loc: str):
    new_loc = str(new_loc)
    matches = hf.get_match_df().set_index("full_key", drop=False)
    match_to_change = matches.loc[match_key]
    old_loc = match_to_change["location"]
    if old_loc == new_loc:
        print("Skipped moving match as it was already at new location.")
        return
    # Find the other match at the location we want to switch to:
    other_match = matches[(matches["location"] == new_loc) & (matches["start"] == match_to_change["start"]) & (matches["sport"] == match_to_change["sport"])]
    matches.loc[match_key, "location"] = new_loc
    print(f"Moved {match_key} from loc {old_loc} to {new_loc}.")
    if len(other_match) == 0:
        return
    matches.loc[other_match.iloc[0]["full_key"], "location"] = old_loc
    print(f"To account for this, moved {other_match.iloc[0]["full_key"]} from {new_loc} to {old_loc}.")
    write_match_backup_from_df(matches, True)

# change_match_location("beer_pongA: 3B: 2", "2")
change_match_location("ping_pongB: 12C: 04", "1")

matches = hf.get_matches()
# Filter for all combinations that happen back-to-back:
back_to_back = [comb for comb in combinations(matches, 2) if (comb[0].end == comb[1].start) and (len(set(comb[0].involved_players).intersection(comb[1].involved_players))>0)]
# filter away sports where this isn't relevant
back_to_back = [comb for comb in back_to_back if comb[0].sport not in ["basketball", "chess", "tennis", "football", "capture_the_flag"]]
# Filter away combinations where the locations already match:
back_to_back = [comb for comb in back_to_back if comb[0].location != comb[1].location]
match_keys = [str((comb[0].match_key, comb[0].location, comb[1].match_key, comb[1].location)) for comb in back_to_back]
print("\n".join(match_keys))
match_df = hf.get_match_df().set_index("full_key", drop=False)
match_df[match_df["sport"] == "ping_pong"].loc[[x.match_key for comb in back_to_back for x in comb]]
# change_match_location('spikeballC: 1A: 1', '1')
# print(back_to_back)


Moved ping_pongB: 12C: 04 from loc 3 to 1.
To account for this, moved ping_pongB: 08C: 09 from 1 to 3.
('ping_pongB: 01C: 13', '1', 'ping_pongA: 13B: 01', '3')


,sport,team_a,team_b,location,day,time,result,winner,start,duration,team_a_key,team_b_key,full_key
full_key,,,,,,,,,,,,,
ping_pongB: 01C: 13,ping_pong,B: Real Mouse,C: Clever Wombat,1,Tuesday,19:45,NaN,NaN,2024-04-30 19:45:00,1800,B: 01,C: 13,ping_pongB: 01C: 13
ping_pongA: 13B: 01,ping_pong,A: Dishonest Fangtooth,B: Real Mouse,3,Tuesday,20:15,NaN,NaN,2024-04-30 20:15:00,1800,A: 13,B: 01,ping_pongA: 13B: 01
